--- Day 20: Pulse Propagation ---
With your help, the Elves manage to find the right parts and fix all of the machines. Now, they just need to send the command to boot up the machines and get the sand flowing again.

The machines are far apart and wired together with long cables. The cables don't connect to the machines directly, but rather to communication modules attached to the machines that perform various initialization tasks and also act as communication relays.

Modules communicate using pulses. Each pulse is either a high pulse or a low pulse. When a module sends a pulse, it sends that type of pulse to each module in its list of destination modules.

There are several different types of modules:

Flip-flop modules (prefix %) are either on or off; they are initially off. If a flip-flop module receives a high pulse, it is ignored and nothing happens. However, if a flip-flop module receives a low pulse, it flips between on and off. If it was off, it turns on and sends a high pulse. If it was on, it turns off and sends a low pulse.

Conjunction modules (prefix &) remember the type of the most recent pulse received from each of their connected input modules; they initially default to remembering a low pulse for each input. When a pulse is received, the conjunction module first updates its memory for that input. Then, if it remembers high pulses for all inputs, it sends a low pulse; otherwise, it sends a high pulse.

There is a single broadcast module (named broadcaster). When it receives a pulse, it sends the same pulse to all of its destination modules.

Here at Desert Machine Headquarters, there is a module with a single button on it called, aptly, the button module. When you push the button, a single low pulse is sent directly to the broadcaster module.

After pushing the button, you must wait until all pulses have been delivered and fully handled before pushing it again. Never push the button if modules are still processing pulses.

Pulses are always processed in the order they are sent. So, if a pulse is sent to modules a, b, and c, and then module a processes its pulse and sends more pulses, the pulses sent to modules b and c would have to be handled first.

The module configuration (your puzzle input) lists each module. The name of the module is preceded by a symbol identifying its type, if any. The name is then followed by an arrow and a list of its destination modules. For example:

broadcaster -> a, b, c
%a -> b
%b -> c
%c -> inv
&inv -> a
In this module configuration, the broadcaster has three destination modules named a, b, and c. Each of these modules is a flip-flop module (as indicated by the % prefix). a outputs to b which outputs to c which outputs to another module named inv. inv is a conjunction module (as indicated by the & prefix) which, because it has only one input, acts like an inverter (it sends the opposite of the pulse type it receives); it outputs to a.

By pushing the button once, the following pulses are sent:

button -low-> broadcaster
broadcaster -low-> a
broadcaster -low-> b
broadcaster -low-> c
a -high-> b
b -high-> c
c -high-> inv
inv -low-> a
a -low-> b
b -low-> c
c -low-> inv
inv -high-> a
After this sequence, the flip-flop modules all end up off, so pushing the button again repeats the same sequence.

Here's a more interesting example:

broadcaster -> a
%a -> inv, con
&inv -> b
%b -> con
&con -> output
This module configuration includes the broadcaster, two flip-flops (named a and b), a single-input conjunction module (inv), a multi-input conjunction module (con), and an untyped module named output (for testing purposes). The multi-input conjunction module con watches the two flip-flop modules and, if they're both on, sends a low pulse to the output module.

Here's what happens if you push the button once:

button -low-> broadcaster
broadcaster -low-> a
a -high-> inv
a -high-> con
inv -low-> b
con -high-> output
b -high-> con
con -low-> output
Both flip-flops turn on and a low pulse is sent to output! However, now that both flip-flops are on and con remembers a high pulse from each of its two inputs, pushing the button a second time does something different:

button -low-> broadcaster
broadcaster -low-> a
a -low-> inv
a -low-> con
inv -high-> b
con -high-> output
Flip-flop a turns off! Now, con remembers a low pulse from module a, and so it sends only a high pulse to output.

Push the button a third time:

button -low-> broadcaster
broadcaster -low-> a
a -high-> inv
a -high-> con
inv -low-> b
con -low-> output
b -low-> con
con -high-> output
This time, flip-flop a turns on, then flip-flop b turns off. However, before b can turn off, the pulse sent to con is handled first, so it briefly remembers all high pulses for its inputs and sends a low pulse to output. After that, flip-flop b turns off, which causes con to update its state and send a high pulse to output.

Finally, with a on and b off, push the button a fourth time:

button -low-> broadcaster
broadcaster -low-> a
a -low-> inv
a -low-> con
inv -high-> b
con -high-> output
This completes the cycle: a turns off, causing con to remember only low pulses and restoring all modules to their original states.

To get the cables warmed up, the Elves have pushed the button 1000 times. How many pulses got sent as a result (including the pulses sent by the button itself)?

In the first example, the same thing happens every time the button is pushed: 8 low pulses and 4 high pulses are sent. So, after pushing the button 1000 times, 8000 low pulses and 4000 high pulses are sent. Multiplying these together gives 32000000.

In the second example, after pushing the button 1000 times, 4250 low pulses and 2750 high pulses are sent. Multiplying these together gives 11687500.

Consult your module configuration; determine the number of low pulses and high pulses that would be sent after pushing the button 1000 times, waiting for all pulses to be fully handled after each push of the button. What do you get if you multiply the total number of low pulses sent by the total number of high pulses sent?

In [40]:
puzzle_input = open("./puzzle_inputs/day20test1.txt").read().split("\n")
puzzle_input

['broadcaster -> a, b, c', '%a -> b', '%b -> c', '%c -> inv', '&inv -> a']

In [41]:
statements = []
for line in puzzle_input:
    statements.append(
        (
            line.split("->")[0].strip(),
            [s.strip() for s in line.split("->")[1].strip().split(",")],
        )
    )

statements

[('broadcaster', ['a', 'b', 'c']),
 ('%a', ['b']),
 ('%b', ['c']),
 ('%c', ['inv']),
 ('&inv', ['a'])]

In [154]:
# Define classes for all nodes in the network:

network = list
network_graph = dict


class Node:
    def __init__(
        self, name: str, outputs: list[str], node_type: str, is_on: bool = False
    ):
        self.name = name
        self.outputs = outputs
        self.node_type = node_type
        self.is_on = is_on

    def __str__(self) -> str:
        message = f"{self.__class__.__name__}: '{self.name}' -> {self.outputs}, type '{self.node_type}', is_on ='{self.is_on}'"
        return message

    def sendPulse(self, pulse: str):
        for node_name in self.outputs:
            network_graph[node_name].recievePulse(pulse, self.name)


""" 
Lets create child classes for Flipflops and Conjunctions.
"""


class FlipFlop(Node):
    def __init__(
        self,
        name: str,
        outputs: list[str],
        node_type: str = "%",
        is_on: bool = False,
    ):
        if node_type != "%":
            raise Exception(f"Incorrect typing for the FlipFlop node '{self.name}'")

        super().__init__(name, outputs, node_type, is_on)

    def __str__(self) -> str:
        return super().__str__()

    def recievePulse(self, pulse: str, dummy_name: str):
        """
        If a flip-flop module receives a high pulse, it is ignored and nothing happens.
        However, if a flip-flop module receives a low pulse, it flips between on and off.

        If it was off, it turns on and sends a high pulse.
        If it was on, it turns off and sends a low pulse.
        """

        if pulse == "High":
            return

        # Send a pulse to outputs
        if self.is_on:
            for node_name in self.outputs:
                network_graph[node_name].recievePulse(pulse="High", node_name=self.name)

        else:
            for node_name in self.outputs:
                network_graph[node_name].recievePulse(pulse="Low", node_name=self.name)

        # Toggle the node
        self.is_on = not self.is_on

    def sendPulse(self, pulse: str):
        return super().sendPulse(pulse)


class Conjunction(Node):
    def __init__(
        self,
        name: str,
        outputs: list[str],
        inputs: list[str],
        input_memory: list[str] = None,
        node_type: str = "&",
        is_on: bool = False,
    ):
        if node_type != "&":
            raise Exception(f"Incorrect typing for the Conjunction node '{self.name}'")

        super().__init__(name, outputs, node_type, is_on)
        self.inputs = inputs

        if input_memory == None:
            self.input_memory = {name: "Low" for name in inputs}

    def __str__(self) -> str:
        return f"{self.__class__.__name__}: {self.inputs} -> '{self.name}' -> {self.outputs}, type '{self.node_type}', is_on ='{self.is_on}'"

    def sendPulse(self, pulse: str):
        return super().sendPulse(pulse)

    def recievePulse(self, pulse: str, node_name: str):
        """
        Conjunction modules remember the type of the most recent pulse received from each of their connected input
        modules; they initially default to remembering a low pulse for each input.

        When a pulse is received, the conjunction module first updates its memory for that input.
        Then, if it remembers high pulses for all inputs, it sends a low pulse; otherwise, it sends a high pulse.
        """

        self.input_memory[node_name] = pulse

        if all([pulse == "High" for pulse in self.input_memory.values()]):
            self.sendPulse(pulse="Low")
        else:
            self.sendPulse(pulse="High")


# class Broadcaster(Node):
#     def __init__(
#         self, name: str, outputs: list[str], node_type: str = "b", is_on: bool = False
#     ):
#         super().__init__(name, outputs, node_type, is_on)

#     def sendPulse(self, pulse: str):
#         return super().sendPulse(pulse)

#     def __str__(self):
#         return super().__str__()

In [155]:
print(FlipFlop(name="F", outputs=["A"]))
print(Conjunction(name="F", outputs=["A"], inputs=["C"]))
print(Broadcaster(name="roadcaster", outputs=["A", "B", "C"]))

FlipFlop: 'F' -> ['A'], type '%', is_on ='False'
Conjunction: ['C'] -> 'F' -> ['A'], type '&', is_on ='False'
Broadcaster: 'roadcaster' -> ['A', 'B', 'C'], type 'b', is_on ='False'


In [130]:
# Create the list of nodes according to the rules:
network = []
for statement in statements:
    network.append(
        Node(name=statement[0][1:], outputs=statement[1], node_type=statement[0][0])
    )

""" 
The trouble is that Conjunction nodes need to have a list of their inputs as well as outputs
"""

# Transform the nodes into their respective types: also create the network_graph
network_graph = {}
for node in network:
    if node.node_type == "%":
        network_graph[node.name] = FlipFlop(node.name, node.outputs)

    if node.node_type == "&":
        # Find all nodes that point to the Conjuntion
        inputs = []
        for n in network:
            if node.name in n.outputs:
                inputs.append(n.name)

        network_graph[node.name] = Conjunction(node.name, node.outputs, inputs)

    if node.node_type == "b":
        network_graph[node.name] = Broadcaster(node.name, node.outputs)

In [131]:
network_graph

{'roadcaster': <__main__.Broadcaster at 0x1063bb280>,
 'a': <__main__.FlipFlop at 0x1063bbca0>,
 'b': <__main__.FlipFlop at 0x1063bbbb0>,
 'c': <__main__.FlipFlop at 0x1063bbd90>,
 'inv': <__main__.Conjunction at 0x1063bbc40>}

In [136]:
network_graph["roadcaster"].sendPulse("High")

In [137]:
[print(n) for n in network_graph.values()]

Broadcaster: 'roadcaster' -> ['a', 'b', 'c'], type 'b', is_on ='False'
FlipFlop: 'a' -> ['b'], type '%', is_on ='False'
FlipFlop: 'b' -> ['c'], type '%', is_on ='False'
FlipFlop: 'c' -> ['inv'], type '%', is_on ='False'
Conjunction: ['c'] -> 'inv' -> ['a'], type '&', is_on ='False'


[None, None, None, None, None]

In [200]:
# Try the same with the second test

puzzle_input = open("./puzzle_inputs/day20test2.txt").read().split("\n")

statements = []
for line in puzzle_input:
    statements.append(
        (
            line.split("->")[0].strip(),
            [s.strip() for s in line.split("->")[1].strip().split(",")],
        )
    )

# Create the list of nodes according to the rules:
network = []
for statement in statements:
    network.append(
        Node(name=statement[0][1:], outputs=statement[1], node_type=statement[0][0])
    )

""" 
The trouble is that Conjunction nodes need to have a list of their inputs as well as outputs
"""
[print(n) for n in network]

Broadcaster: 'roadcaster' -> ['a'], type 'b', is_on ='False'
FlipFlop: 'a' -> ['inv', 'con'], type '%', is_on ='False'
Conjunction: [] -> 'inv' -> ['b'], type '&', is_on ='False'
FlipFlop: 'b' -> ['con'], type '%', is_on ='False'
Conjunction: [] -> 'con' -> ['output'], type '&', is_on ='False'


[None, None, None, None, None]

In [201]:
# Transform the nodes into their respective types: also create the network_graph
network_graph = {}
for node in network:
    if node.node_type == "&":
        # Find all nodes that point to the Conjuntion
        inputs = []
        for n in network:
            if node.name in n.outputs:
                inputs.append(n.name)

        network_graph[node.name] = Node(
            name=node.name,
            node_type=node.node_type,
            outputs=node.outputs,
            inputs=inputs,
        )

    else:
        network_graph[node.name] = node

[print(f"{name} : {node}") for name, node in network_graph.items()]

roadcaster : Broadcaster: 'roadcaster' -> ['a'], type 'b', is_on ='False'
a : FlipFlop: 'a' -> ['inv', 'con'], type '%', is_on ='False'
inv : Conjunction: ['a'] -> 'inv' -> ['b'], type '&', is_on ='False'
b : FlipFlop: 'b' -> ['con'], type '%', is_on ='False'
con : Conjunction: ['a', 'b'] -> 'con' -> ['output'], type '&', is_on ='False'


[None, None, None, None, None]

In [202]:
network_graph["roadcaster"].sendPulse("Low")
[print(n) for n in network_graph.values()]

KeyError: 'output'

In [ ]:
# TODO: This is a mess DONE !
# I need to unify the recieve pulse function to handle the Conjunction and FlipFlop cases

""" 
For that:
    All nodes need to have a input list
    The typing will only matter in the recievePulse method, there I can nestle the logic
for both cases + broadcaster

    re-do the class

"""

In [277]:
# Re-implementation:
pulses_sent = {"High": 0, "Low": 0}

network_graph = {}


class Node:
    def __init__(
        self,
        name: str,
        outputs: list[str],
        node_type: str,
        inputs: list[str] = [],
        is_on: bool = False,
    ):
        self.name = name
        self.outputs = outputs
        self.inputs = inputs
        self.node_type = node_type
        self.is_on = is_on

        self.input_memory = {name: "Low" for name in inputs}

    def __str__(self) -> str:
        key_name = {
            "%": "FlipFlop",
            "&": "Conjunction",
            "b": "Broadcaster",
            "dummy": "dummy",
        }

        if self.node_type == "&":
            return f"{key_name[self.node_type]}: {self.inputs} -> '{self.name}' -> {self.outputs}, type '{self.node_type}', is_on ='{self.is_on}'"

        else:
            return f"{key_name[self.node_type]}: '{self.name}' -> {self.outputs}, type '{self.node_type}', is_on ='{self.is_on}'"

    def sendPulse(self, pulse: str):
        for node_name in self.outputs:
            pulses_sent[pulse] += 1
            network_graph[node_name].recievePulse(pulse, self.name)

    def recievePulse(self, pulse: str, emitter: str):

        if self.node_type == "%":
            """
            If a flip-flop module receives a high pulse, it is ignored and nothing happens.
            However, if a flip-flop module receives a low pulse, it flips between on and off.

            If it was off, it turns on and sends a high pulse.
            If it was on, it turns off and sends a low pulse.
            """
            if pulse == "High":
                return
            # Send a pulse to outputs
            if self.is_on:
                for node_name in self.outputs:
                    network_graph[node_name].recievePulse(
                        pulse="Low", emitter=self.name
                    )

            else:
                for node_name in self.outputs:
                    network_graph[node_name].recievePulse(
                        pulse="High", emitter=self.name
                    )

            # Toggle the node
            self.is_on = not self.is_on

        if self.node_type == "&":
            """
            Conjunction modules remember the type of the most recent pulse received from each of their connected input
            modules; they initially default to remembering a low pulse for each input.

            When a pulse is received, the conjunction module first updates its memory for that input.
            Then, if it remembers high pulses for all inputs, it sends a low pulse; otherwise, it sends a high pulse.
            """

            self.input_memory[emitter] = pulse

            if all([p == "High" for p in self.input_memory.values()]):
                self.sendPulse(pulse="Low")
            else:
                self.sendPulse(pulse="High")

        return

In [278]:
# Try the same with the second test

puzzle_input = open("./puzzle_inputs/day20test1.txt").read().split("\n")
pulses_sent = {"High": 0, "Low": 0}

statements = []
for line in puzzle_input:
    statements.append(
        (
            line.split("->")[0].strip(),
            [s.strip() for s in line.split("->")[1].strip().split(",")],
        )
    )

# Create the list of nodes according to the rules:
network = []
for statement in statements:
    network.append(
        Node(name=statement[0][1:], outputs=statement[1], node_type=statement[0][0])
    )

""" 
The trouble is that Conjunction nodes need to have a list of their inputs as well as outputs
"""
# [print(n) for n in network]

# Transform the nodes into their respective types: also create the network_graph
network_graph = {}
for node in network:
    if node.node_type == "&":
        # Find all nodes that point to the Conjuntion
        inputs = []
        for n in network:
            if node.name in n.outputs:
                inputs.append(n.name)

        network_graph[node.name] = Node(
            name=node.name,
            node_type=node.node_type,
            outputs=node.outputs,
            inputs=inputs,
        )

    else:
        network_graph[node.name] = node

# Add extra nodes for outputs when generating the graph. Fill them with dummy nodes:
dummy_nodes = []
for name, node in network_graph.items():
    for output in node.outputs:
        if output not in network_graph.keys():
            dummy_nodes.append(output)

for dn in dummy_nodes:
    network_graph[output] = Node(name=output, outputs=[], node_type="dummy")


[print(f"{name} : {node}") for name, node in network_graph.items()]
print(pulses_sent)

roadcaster : Broadcaster: 'roadcaster' -> ['a', 'b', 'c'], type 'b', is_on ='False'
a : FlipFlop: 'a' -> ['b'], type '%', is_on ='False'
b : FlipFlop: 'b' -> ['c'], type '%', is_on ='False'
c : FlipFlop: 'c' -> ['inv'], type '%', is_on ='False'
inv : Conjunction: ['c'] -> 'inv' -> ['a'], type '&', is_on ='False'
{'High': 0, 'Low': 0}


In [279]:
# Send a pulse and see its effect
for __ in range(1):
    network_graph["roadcaster"].sendPulse("Low")
    # network_graph["roadcaster"].sendPulse("High")
[print(n) for n in network_graph.values()]
print(pulses_sent)

""" 
There is a problem when construtiong the graph:
    Some nodes do not appear the list, because they are not sending to any other node.

    Solution 1:
        add exeptions to every call of the network_graph

        if node_name in self.outputs:
        -> if node_name in node_graph.keys() <-
            node_graph[node_name].recievePulse( ...

    Solution 2:
        add extra nodes for outputs when generating the graph. Fill them with dummy nodes:

        for name, node in network_graph.items():
            for output in node.outputs:
                if output not in network_graph.keys():
                    network_graph[output] = Node(
                        name = output,
                        outputs = [],
                        node_type = "%"
                    )

I decided to implement soution 2

"""

RecursionError: maximum recursion depth exceeded in comparison

In [270]:
# Now we add a counter to keep track of the node activations

print(pulses_sent)

{'High': 0, 'Low': 1192}
